Note: The script uses Berkeley Neural Parser to parse the generated instructions, and visualize the results using Plotly.

Please make sure to install benepar following their documentation [here](https://github.com/nikitakit/self-attentive-parser#installation).

In [2]:
import benepar, spacy
nlp = spacy.load('en_core_web_md')
doc = nlp("The time for action is now. It's never too late to do something.")

if spacy.__version__.startswith('2'):
    nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
    nlp.add_pipe("benepar", config={"model": "benepar_en3"})

In [3]:
def find_root_verb_and_its_dobj(tree_root):
    # first check if the current node and its children satisfy the condition
    if tree_root.pos_ == "VERB":
        for child in tree_root.children:
            if child.dep_ == "dobj" and child.pos_ == "NOUN":
                return tree_root.lemma_, child.lemma_
        return tree_root.lemma_, None
    # if not, check its children
    for child in tree_root.children:
        return find_root_verb_and_its_dobj(child)
    # if no children satisfy the condition, return None
    return None, None

def find_root_verb_and_its_dobj_in_string(s):
    doc = nlp(s)
    first_sent = list(doc.sents)[0]
    return find_root_verb_and_its_dobj(first_sent.root)

find_root_verb_and_its_dobj_in_string("Write me a story about education.")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/uj-user/Yo/torch-venv/lib/python3.8/site-packages/torch/distributions/distribution.py:51: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


('write', 'story')

In [4]:
import pandas as pd
import json
import tqdm

model =   'google/flan-t5-base'
category = 'open_qa'
df = pd.read_csv(f'./d_{model}/{category}.csv')
data = df['0'].dropna().tolist()
print(len(data))

# generated_data_path = "../data/gpt3_generations/machine_generated_instructions.jsonl" # replace this with your own data path
# machine_generated_tasks = []
# with open(generated_data_path) as fin:
#     for line in fin:
#         machine_generated_tasks.append(json.loads(line))

# instructions = set([task["instruction"] for task in machine_generated_tasks])
# print(len(instructions))

raw_phrases = []
for instruction in tqdm.tqdm(data):
    try:
        verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
        raw_phrases.append({
            "verb": verb,
            "noun": noun,
            "instruction": instruction
        })
    except Exception as e:
        print(e)
        print(instruction)

3609


  0%|          | 0/3609 [00:00<?, ?it/s]/home/uj-user/Yo/torch-venv/lib/python3.8/site-packages/torch/distributions/distribution.py:51: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
  1%|          | 34/3609 [00:00<01:08, 52.34it/s]


KeyboardInterrupt: 

In [5]:
len(raw_phrases)

3609

In [6]:
raw_phrases = pd.DataFrame(raw_phrases)
phrases = pd.DataFrame(raw_phrases).dropna()
phrases[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

verb     noun      
eat      lot           5
make     money         3
have     impact        3
take     breath        3
preheat  oven          3
                      ..
have     population    1
         history       1
         eye           1
         dollar        1
win      game          1
Length: 122, dtype: int64

In [38]:
import pandas as pd
import json
import tqdm

model =   'google/flan-t5-base'
category = 'All'
df_model = pd.read_csv(f'./d_{model}/{category}.csv')
data = df_model['0'].dropna().tolist()
print(len(data))

raw_phrases = []
for instruction in tqdm.tqdm(data):
    try:
        verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
        raw_phrases.append({
            "verb": verb,
            "noun": noun,
            "instruction": instruction
        })
    except Exception as e:
        print(e)
        print(instruction)
        
raw_phrases_b = pd.DataFrame(raw_phrases)
phrases_b = pd.DataFrame(raw_phrases_b).dropna()
phrases_b[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

14994


  0%|          | 0/14994 [00:00<?, ?it/s]/home/uj-user/Yo/torch-venv/lib/python3.8/site-packages/torch/distributions/distribution.py:51: UserWarning:

<class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.

100%|██████████| 14994/14994 [06:41<00:00, 37.33it/s]


verb     noun    
eat      lot         26
have     lot         19
make     list        15
preheat  oven        13
drink    glass       13
                     ..
have     fever        1
         finisher     1
         fit          1
         flavor       1
yield    curve        1
Length: 755, dtype: int64

In [40]:
import pandas as pd
import json
import tqdm
import re

model =   'google/flan-t5-base'
category = 'All'
df_model = pd.read_csv(f'./d_{model}/{category}.csv')
data = df_model['0'].dropna().tolist()
print(len(data))

raw_phrases = []
for instruction in tqdm.tqdm(data):
    instruction = re.sub(r"[^a-zA-Z0-9\s]", "", instruction)
    try:
        verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
        raw_phrases.append({
            "verb": verb,
            "noun": noun,
            "instruction": instruction
        })
    except Exception as e:
        print(e)
        print(instruction)
        
raw_phrases_n = pd.DataFrame(raw_phrases)
phrases_n = pd.DataFrame(raw_phrases_n).dropna()
phrases_n[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

14994


  0%|          | 0/14994 [00:00<?, ?it/s]/home/uj-user/Yo/torch-venv/lib/python3.8/site-packages/torch/distributions/distribution.py:51: UserWarning:

<class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.

 15%|█▌        | 2265/14994 [01:01<05:18, 40.02it/s]

list index out of range



 25%|██▌       | 3810/14994 [01:42<04:48, 38.77it/s]

list index out of range



...

list index out of range



 71%|███████▏  | 10714/14994 [04:58<01:43, 41.35it/s]

list index out of range



100%|██████████| 14994/14994 [06:52<00:00, 36.35it/s]


verb     noun        
eat      lot             26
have     lot             18
make     list            16
preheat  oven            14
drink    glass           13
                         ..
have     conductivity     1
         dinner           1
         door             1
         effect           1
yield    curve            1
Length: 775, dtype: int64

In [39]:
import os
import plotly.graph_objects as go
import plotly.express as px

# df["blank"] = "ROOT"
df = phrases_b.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(50).reset_index().rename(columns={0: "count"})

#df = df[df["count"] > 30]
print(df)
fig = px.sunburst(df, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
)
fig.show()
os.makedirs('./output', exist_ok=True)
fig.write_html("./output/verb_noun.html")
# fig.savefig("output/verb_noun.pdf")

       verb           noun  count
0       eat            lot     26
1      have            lot     19
2      make           list     15
3   preheat           oven     13
4     drink          glass     13
5      take         breath     10
6      take            nap     10
7      take          class      9
8      take           walk      9
9      plan           trip      8
10     take         shower      8
11     have          sense      7
12    press         button      7
13     have           rate      6
14      use    screwdriver      6
15    place          steak      6
16     take            bus      6
17     make          money      5
18     have           time      5
19     have         impact      5
20     play           game      5
21     have     population      5
22    place          piece      5
23     gain  understanding      4
24    place            egg      4
25     have          child      3
26  receive          award      3
27     take           test      3
28      add   

In [33]:
# import pandas as pd
# import json
# import tqdm

# model =   'google/t5-v1_1-small'
# category = 'All'
# df_model = pd.read_csv(f'./d_{model}/{category}.csv')
# data = df_model['0'].dropna().tolist()
# print(len(data))

# raw_phrases = []
# for instruction in tqdm.tqdm(data):
#     try:
#         verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
#         raw_phrases.append({
#             "verb": verb,
#             "noun": noun,
#             "instruction": instruction
#         })
#     except Exception as e:
#         print(e)
#         print(instruction)
        
raw_phrases = pd.DataFrame(raw_phrases)
phrases = pd.DataFrame(raw_phrases).dropna()
phrases[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

verb     noun           
have     question           8
do       thing              4
write    story              4
reserve  right              4
extract  date               3
                           ..
fix      slice              1
fit      size               1
find     specific           1
         restaurant         1
’ve      differentiation    1
Length: 533, dtype: int64

In [42]:
import os
import plotly.graph_objects as go
import plotly.express as px

# df["blank"] = "ROOT"
df = phrases.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(30).reset_index().rename(columns={0: "count"})

#df = df[df["count"] > 30]
print(df)
fig = px.sunburst(df, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
)
fig.show()
os.makedirs('./output', exist_ok=True)
fig.write_html("./output/verb_noun.html")
# fig.savefig("output/verb_noun.pdf")

       verb          noun  count
0      have      question      8
1        do         thing      4
2     write         story      4
3   reserve         right      4
4   extract          date      3
5      plan          trip      3
6      make         money      3
7   control      daughter      3
8       eat          post      3
9      lose        weight      3
10     have          team      3
11     have           lot      3
12     play          game      3
13     list          type      2
14     have         child      2
15     join  conversation      2
16     post     permalink      2
17      get          name      2
18     have        number      2
19    score           run      2
20     mean       meaning      2
21     have   information      2
22     have          idea      2
23     know           one      2
24    score          goal      2
25     find          list      2
26    score         point      2
27     have       problem      2
28     pick          name      2
29    writ

In [35]:
# import pandas as pd
# import json
# import tqdm

# model =   'google/flan-t5-small'
# category = 'All'
# df_model = pd.read_csv(f'./d_{model}/{category}.csv')
# data = df_model['0'].dropna().tolist()
# print(len(data))

# raw_phrases_l = []
# for instruction in tqdm.tqdm(data):
#     try:
#         verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
#         raw_phrases_l.append({
#             "verb": verb,
#             "noun": noun,
#             "instruction": instruction
#         })
#     except Exception as e:
#         print(e)
#         print(instruction)
        
raw_phrases_l = pd.DataFrame(raw_phrases_l)
phrases_l = pd.DataFrame(raw_phrases_l).dropna()
phrases_l[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

verb   noun   
use    knife      23
take   breath     15
use    pair       15
have   lot        13
plan   trip       11
                  ..
have   speed       1
       support     1
       value       1
       water       1
yield  curve       1
Length: 756, dtype: int64

In [41]:
import os
import plotly.graph_objects as go
import plotly.express as px

# df["blank"] = "ROOT"
df = phrases_l.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(30).reset_index().rename(columns={0: "count"})

#df = df[df["count"] > 30]
print(df)
fig = px.sunburst(df, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
)
fig.show()
os.makedirs('./output', exist_ok=True)
fig.write_html("./output/verb_noun.html")
# fig.savefig("output/verb_noun.pdf")

       verb         noun  count
0       use        knife     23
1      take       breath     15
2       use         pair     15
3      have          lot     13
4      plan         trip     11
5      make         list     11
6       use   calculator     10
7      take        break      8
8       use  screwdriver      8
9      take          bus      6
10     take          nap      6
11      eat         diet      6
12     take       shower      6
13    place        steak      6
14     play         game      6
15     take       amount      5
16     take         care      5
17  preheat         oven      5
18     list        title      5
19     have       impact      5
20      use    treadmill      4
21    drink        glass      4
22      use       cutter      4
23     take         step      4
24     have         idea      4
25     pour          cup      4
26     need          cup      4
27     take        place      4
28     need          lot      4
29     play        chess      4


In [32]:
# import pandas as pd
# import json
# import tqdm

# model =   'google/flan-t5-large'
# category = 'All'
# df_model = pd.read_csv(f'./d_{model}/{category}.csv')
# data = df_model['0'].dropna().tolist()
# print(len(data))

# raw_phrases_l = []
# for instruction in tqdm.tqdm(data):
#     instruction = re.sub(r"[^a-zA-Z0-9\s]", "", instruction)
#     try:
#         verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
#         raw_phrases_l.append({
#             "verb": verb,
#             "noun": noun,
#             "instruction": instruction
#         })
#     except Exception as e:
#         print(e)
#         print(instruction)
        
# raw_phrases_l = pd.DataFrame(raw_phrases_l)
# phrases_l = pd.DataFrame(raw_phrases_l).dropna()
# phrases_l[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)
# phrases_l.to_csv('flan-t5-large-verb-pairs.csv', index=False)

# import os
# import plotly.graph_objects as go
# import plotly.express as px

# df["blank"] = "ROOT"

phrases_l = pd.read_csv('flan-t5-large-verb-pairs.csv')
df = phrases_l.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(50).reset_index().rename(columns={0: "count"})

#df = df[df["count"] > 30]
print(df)
fig = px.sunburst(df, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    title="Flan-T5-Large",
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
    font=dict(size=40),
)

fig.show()
os.makedirs('./output', exist_ok=True)
fig.write_image(f"./output/flan-t5-piechart.svg")

        verb         noun  count
0        get          gun     31
1       have          lot     18
2      drink        glass     12
3        eat        fruit     10
4       play         game      9
5        buy          car      8
6       make         list      8
7       take       breath      7
8       take          bus      6
9       take       shower      6
10     place        steak      5
11       get     exercise      5
12    absorb     sunlight      5
13    reduce       stress      5
14      take         trip      5
15    reduce         risk      5
16      give       advice      5
17       use  screwdriver      4
18       eat          lot      4
19      make        debut      4
20      fill          cup      4
21      have       impact      4
22      take          nap      4
23    reduce       amount      4
24       buy         bike      4
25  consider       factor      4
26      slap         shot      3
27   improve      quality      3
28      take         time      3
29      fi

In [33]:
# import pandas as pd
# import json
# import tqdm

# model =   'google/t5-v1_1-large'
# category = 'All'
# df_model = pd.read_csv(f'./d_{model}/{category}.csv')
# data = df_model['0'].dropna().tolist()
# print(len(data))

# raw_phrases_l = []
# count = 0
# for instruction in tqdm.tqdm(data):
#     instruction = re.sub(r"[^a-zA-Z0-9\s]", "", instruction)
#     try:
#         verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
#         raw_phrases_l.append({
#             "verb": verb,
#             "noun": noun,
#             "instruction": instruction
#         })
#     except Exception as e:
#         #print(e)
#         #print(instruction)
#         count +=1
# print('error count:', count)
# raw_phrases_l = pd.DataFrame(raw_phrases_l)
# phrases_l = pd.DataFrame(raw_phrases_l).dropna()
# phrases_l[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)
# phrases_l.to_csv('t5-large-verb-pairs.csv', index=False)

# import os
# import plotly.graph_objects as go
# import plotly.express as px

# df["blank"] = "ROOT"

phrases_l = pd.read_csv('t5-large-verb-pairs.csv')
df = phrases_l.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(50).reset_index().rename(columns={0: "count"})

#df = df[df["count"] > 30]
print(df)
fig = px.sunburst(df, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    title="T5-Large",
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
    font=dict(size=40)
)

fig.show()
os.makedirs('./output', exist_ok=True)
fig.write_image(f"./output/t5-piechart.svg")

        verb          noun  count
0    explain        answer     13
1    extract          name      8
2        win         title      7
3     answer      question      7
4       take         place      6
5      begin        career      4
6       list          name      4
7       play          game      4
8       have       history      4
9      write         point      4
10    answer          none      4
11   provide          list      4
12       win  championship      4
13      give       example      3
14      have        people      3
15       use        stream      3
16     write         essay      3
17  classify     following      3
18      sign      contract      3
19    answer            mg      3
20   provide       summary      3
21     write          name      3
22      play          role      3
23   extract           way      2
24   receive         award      2
25       use       passage      2
26      have          user      2
27       buy       bicycle      2
28       use  

In [30]:
import pandas as pd
import json
import tqdm
import re

model =   'google/flan-t5-large'
category = 'All'
df_model = pd.read_csv(f'./d_{model}/{category}.csv')
data = df_model['0'].dropna().tolist()
print(len(data))

raw_phrases_l = []
for instruction in tqdm.tqdm(data):
    instruction = re.sub(r"[^a-zA-Z0-9\s]", "", instruction)
    try:
        verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
        raw_phrases_l.append({
            "verb": verb,
            "noun": noun,
            "instruction": instruction
        })
        print({
            "verb": verb,
            "noun": noun,
            "instruction": instruction}
        )

    except Exception as e:
        print(e)
        print(instruction)
        

15010


  0%|          | 0/15010 [00:00<?, ?it/s]/home/uj-user/Yo/torch-venv/lib/python3.8/site-packages/torch/distributions/distribution.py:51: UserWarning:

<class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.

  0%|          | 6/15010 [00:00<04:56, 50.56it/s]

{'verb': None, 'noun': None, 'instruction': '31 August 2000'}
{'verb': None, 'noun': None, 'instruction': 'tope'}
{'verb': 'have', 'noun': 'rate', 'instruction': 'camels have a high metabolic rate'}
{'verb': None, 'noun': None, 'instruction': 'sarah'}
{'verb': None, 'noun': None, 'instruction': 'July 10 1981'}
{'verb': 'stalemate', 'noun': 'result', 'instruction': 'Stalemate results in a draw'}
{'verb': None, 'noun': None, 'instruction': 'Grant Park in Chicago'}
{'verb': None, 'noun': None, 'instruction': 'george washington'}
{'verb': None, 'noun': None, 'instruction': 'mobile is bad for human'}
{'verb': None, 'noun': None, 'instruction': 'John Moses Browning'}
{'verb': 'refer', 'noun': None, 'instruction': 'Thomas Jefferson may refer to'}


  0%|          | 18/15010 [00:00<04:55, 50.81it/s]

{'verb': None, 'noun': None, 'instruction': 'Boland U21'}
{'verb': None, 'noun': None, 'instruction': 'Dar es Salaam Region'}
{'verb': None, 'noun': None, 'instruction': 'isosceles'}
{'verb': 'take', 'noun': 'shower', 'instruction': 'take a shower'}
{'verb': None, 'noun': None, 'instruction': 'A Song of Ice and Fire'}
{'verb': None, 'noun': None, 'instruction': 'Process mining is a family of techniques'}
{'verb': 'make', 'noun': None, 'instruction': 'Tie backs can be made out of tin foil'}
{'verb': 'use', 'noun': None, 'instruction': 'is used to disperse light'}
{'verb': None, 'noun': None, 'instruction': 'Cantaro'}
{'verb': None, 'noun': None, 'instruction': 'ping'}
{'verb': None, 'noun': None, 'instruction': 'nelson mandela'}


...

 14%|█▎        | 2054/15010 [00:40<04:37, 46.77it/s]

{'verb': None, 'noun': None, 'instruction': 'kings of leon'}
{'verb': None, 'noun': None, 'instruction': 'The first commandment is that a president should respect the authority of Congress'}
{'verb': 'serve', 'noun': 'variety', 'instruction': 'In the United States political parties have served a variety of functions in American politics'}
{'verb': None, 'noun': None, 'instruction': 'Lyndon Baines Johnson'}
{'verb': None, 'noun': None, 'instruction': '1971'}
{'verb': None, 'noun': None, 'instruction': 'matter and energy'}
{'verb': 'use', 'noun': None, 'instruction': 'knots are used to fasten things together'}
{'verb': None, 'noun': None, 'instruction': '6 April 2003'}
{'verb': 'want', 'noun': None, 'instruction': 'For the main course youll want to make a turkey and stuffing entree Make sure there are options for vegetarians or people with Celiac disease in each of these courses'}
{'verb': None, 'noun': None, 'instruction': 'irrational'}


 14%|█▎        | 2063/15010 [00:40<04:13, 51.10it/s]


{'verb': None, 'noun': None, 'instruction': '292'}
{'verb': None, 'noun': None, 'instruction': 'Brazil was the leading grower of coffee beans producing 35'}
{'verb': None, 'noun': None, 'instruction': 'more efficient access to data'}
{'verb': 'help', 'noun': None, 'instruction': 'Can solar panels help maintain power during a storm'}
{'verb': 'consider', 'noun': 'factor', 'instruction': 'You should consider the following factors when choosing a dog'}
{'verb': None, 'noun': None, 'instruction': 'Stephanie Izard'}
{'verb': 'act', 'noun': None, 'instruction': 'Gravity always acts in attraction drawing two masses together'}
{'verb': None, 'noun': None, 'instruction': 'cacti'}


KeyboardInterrupt: 

In [6]:
import pandas as pd
import json
import tqdm
import re

model =   'google/flan-t5-large'
category = 'All'
df_model = pd.read_csv(f'./d_{model}/{category}.csv')
data = df_model['0'].dropna().tolist()
print(len(data))

raw_phrases_l = []
for instruction in tqdm.tqdm(data):
    instruction = re.sub(r"[^a-zA-Z0-9\s]", "", instruction)
    print(instruction)
    try:
        verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
        raw_phrases_l.append({
            "verb": verb,
            "noun": noun,
            "instruction": instruction
        })
        
    except Exception as e:
        pass
        # print(e)
        # print(instruction)
        
raw_phrases_l = pd.DataFrame(raw_phrases_l)
phrases_l = pd.DataFrame(raw_phrases_l).dropna()
phrases_l[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)
phrases_l.to_csv('flan-t5-large-verb-pairs-new.csv', index=False)

15011


100%|██████████| 15011/15011 [00:00<00:00, 195929.89it/s]

31 August 2000
tope
camels have a high metabolic rate
sarah
July 10 1981
Stalemate results in a draw
Grant Park in Chicago
george washington
mobile is bad for human
John Moses Browning
Thomas Jefferson may refer to
Boland U21
Dar es Salaam Region
isosceles
take a shower
A Song of Ice and Fire
Process mining is a family of techniques
Tie backs can be made out of tin foil
is used to disperse light
Cantaro
ping
nelson mandela
johannes van gogh
it sets
eat
Lamborghini was founded in 1963 to compete with Ferrari
bicycle
stringed instrument
Golden Retrievers have an average lifespan of 12 to 13 years
willem iv
Xbox 360
johnny depp
he did not want to be part of band
REST API
underwriting
A good coffee shop should have a wide variety of food and drinks
LinkedIn is a business and employmentfocused social media platform
andromeda
You can donate to Spark by clicking the button below
aristocrats
In 2014 American journalist TaNehisi Coates published an article titled The Case for Reparations which 

KeyError: "None of [Index(['verb', 'noun'], dtype='object')] are in the [columns]"

In [3]:
!pip install re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
